<a href="https://colab.research.google.com/github/sejalvyas13/Covid-19QnA/blob/master/Covid_19QnA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load Dataset

In [0]:
from google.colab import drive
drive.mount('MyDrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at MyDrive


In [0]:
from pathlib import PurePath
import pandas as pd

input_dir = PurePath('MyDrive/My Drive/CORD-19-research-challenge')
metadata_path = input_dir / 'metadata.csv'
metadata_df = pd.read_csv(metadata_path)
metadata_df = metadata_df.dropna(subset=['abstract', 'title']).reset_index(drop = True)

**Search Engine**

Searches for top N contexts from all the abstracts

In [0]:
!pip install rank_bm25

In [0]:
from rank_bm25 import BM25Okapi


def searchEngine(query, num) :
  abstracts = metadata_df["abstract"].tolist()

  tokenized_abstracts = [doc.split(" ") for doc in abstracts]

  bm25  = BM25Okapi(tokenized_abstracts)

  tokenized_query = query.split(" ")

  return bm25.get_top_n(tokenized_query, abstracts, n = num)




**Pretrained BERT**

In [0]:
!pip install transformers

In [0]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

model = model.to(torch_device)

In [0]:
def getAnswer(question, context) :
  encoded_dict = tokenizer.encode_plus(question, 
                                      context,
                                      add_special_tokens = True,
                                      max_length = 256,
                                      pad_to_max_length = True,
                                      return_tensors = 'pt')
  input_ids = encoded_dict['input_ids'].to(torch_device)
  token_type_ids = encoded_dict['token_type_ids'].to(torch_device)

  start_scores, end_scores = model(input_ids, token_type_ids=token_type_ids)

  all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
  start_index = torch.argmax(start_scores)
  end_index = torch.argmax(end_scores)
  answer = tokenizer.convert_tokens_to_string(all_tokens[start_index:end_index+1])
  answer = answer.replace('[CLS]', '')
  answer = answer.replace('[SEP]', '')
  return answer


**All the Kaggle questions**

In [0]:
covid_kaggle_questions = {
"data":[
          {
              "task": "What is known about transmission, incubation, and environmental stability?",
              "questions": [
                  "Is the virus transmitted by aerisol, droplets, food, close contact, fecal matter, or water?",
                  "How long is the incubation period for the virus?",
                  "Can the virus be transmitted asymptomatically or during the incubation period?",
                  "How does weather, heat, and humidity affect the tramsmission of 2019-nCoV?",
                  "How long can the 2019-nCoV virus remain viable on common surfaces?"
              ]
          },
          {
              "task": "What do we know about COVID-19 risk factors?",
              "questions": [
                  "What risk factors contribute to the severity of 2019-nCoV?",
                  "How does hypertension affect patients?",
                  "How does heart disease affect patients?",
                  "How does copd affect patients?",
                  "How does smoking affect patients?",
                  "How does pregnancy affect patients?",
                  "What is the fatality rate of 2019-nCoV?",
                  "What public health policies prevent or control the spread of 2019-nCoV?"
              ]
          },
          {
              "task": "What do we know about virus genetics, origin, and evolution?",
              "questions": [
                  "Can animals transmit 2019-nCoV?",
                  "What animal did 2019-nCoV come from?",
                  "What real-time genomic tracking tools exist?",
                  "What geographic variations are there in the genome of 2019-nCoV?",
                  "What efforts are being done in asia to prevent further outbreaks?"
              ]
          },
          {
              "task": "What do we know about vaccines and therapeutics?",
              "questions": [
                  "What drugs or therapies are being investigated?",
                  "Are anti-inflammatory drugs recommended?"
              ]
          },
          {
              "task": "What do we know about non-pharmaceutical interventions?",
              "questions": [
                  "Which non-pharmaceutical interventions limit tramsission?",
                  "What are most important barriers to compliance?"
              ]
          },
          {
              "task": "What has been published about medical care?",
              "questions": [
                  "How does extracorporeal membrane oxygenation affect 2019-nCoV patients?",
                  "What telemedicine and cybercare methods are most effective?",
                  "How is artificial intelligence being used in real time health delivery?",
                  "What adjunctive or supportive methods can help patients?"
              ]
          },
          {
              "task": "What do we know about diagnostics and surveillance?",
              "questions": [
                  "What diagnostic tests (tools) exist or are being developed to detect 2019-nCoV?"
              ]
          },
          {
              "task": "Other interesting questions",
              "questions": [
                  "What is the immune system response to 2019-nCoV?",
                  "Can personal protective equipment prevent the transmission of 2019-nCoV?",
                  "Can 2019-nCoV infect patients a second time?"
              ]
          }
   ]
}


Function for finding answer to all the questions

In [0]:
def QnASystem(n) :
  ckq = covid_kaggle_questions
  result = []
  ans = []
  obj = {}
  interimResult = []
  for i in ckq["data"] :
    for j in i["questions"] :
      topNContexts = searchEngine(j , n)
      for k in range(n) :
        a = getAnswer(j, topNContexts[k])
        if a.strip() != '' : 
          ans.append(a)
      obj["question"] = j
      obj["answers"] = ans
      ans = []
      interimResult.append(obj)
      obj = {}
    result.append({'Task' : i['task'],
                   'results' : interimResult})  
    print("Finding answer to task", i)
    
    
  return result
    
  


In [0]:
results = QnASystem(10)


Finding answer to task {'task': 'What is known about transmission, incubation, and environmental stability?', 'questions': ['Is the virus transmitted by aerisol, droplets, food, close contact, fecal matter, or water?', 'How long is the incubation period for the virus?', 'Can the virus be transmitted asymptomatically or during the incubation period?', 'How does weather, heat, and humidity affect the tramsmission of 2019-nCoV?', 'How long can the 2019-nCoV virus remain viable on common surfaces?']}
Finding answer to task {'task': 'What do we know about COVID-19 risk factors?', 'questions': ['What risk factors contribute to the severity of 2019-nCoV?', 'How does hypertension affect patients?', 'How does heart disease affect patients?', 'How does copd affect patients?', 'How does smoking affect patients?', 'How does pregnancy affect patients?', 'What is the fatality rate of 2019-nCoV?', 'What public health policies prevent or control the spread of 2019-nCoV?']}
Finding answer to task {'tas

Display all the Questions and Answers

In [0]:
#print results


for i in range(len(results)) : 
  print('TASK :', results[i]['Task'].upper())
  for j in range(len(results[i]['results'])):
    print('Question :', results[i]['results'][j]['question'])        
    print('Answers :', results[i]['results'][j]['answers'])
    print("\n")
  print("\n")
  print("------------------------------------------------------------------------")
  print("\n")



TASK : WHAT IS KNOWN ABOUT TRANSMISSION, INCUBATION, AND ENVIRONMENTAL STABILITY?
Question : Is the virus transmitted by aerisol, droplets, food, close contact, fecal matter, or water?
Answers : ['transmission is via aerosols , droplets , or fomites ( contaminated surfaces ) . environmental factors which are often important for virus survival may include the ambient humidity , temperature , and ph of the environment they are in , so many viruses survive only a few hours in the environment and are often readily inactivated by common hygiene techniques , such as using soap and water , and some detergents , disinfectants , and antiseptics . sexually transmitted viral infections , such as herpes simplex , are often transmitted by close mucosal contacts', 'the virus is transmitted through droplets , close contact , and other means', 'air , water , food , or living vectors', 'close interpersonal contact', ' is the virus transmitted by aerisol , droplets , food , close contact , fecal matter 